In [ ]:
pip install langgraph langchain langchain_community langchainhub langchain_groq langchain_huggingface bs4 tiktoken chromadb

## Brief

In [ ]:
from typing import Annotated, Literal, Sequence, TypedDict
from langchain import hub
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.graph.message import add_messages
from langgraph.prebuilt import tools_condition
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode
from langchain_core.output_parsers import StrOutputParser

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
import os
os.environ["GROQ_API_KEY"] = ''
os.environ['HF_TOKEN'] = ''

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(model_name='Gemma2-9b-It')

In [ ]:
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering"
]

In [ ]:
docs = [WebBaseLoader(url).load() for url in urls]

In [ ]:
docs_list = [item for sublist in docs for item in sublist]
# Convert each dictionary to a Document instance
from langchain.docstore.document import Document

docs_list = [
    Document(page_content=doc["page_content"], metadata=doc.get("metadata", {}))
    if isinstance(doc, dict) else doc
    for doc in docs_list
]

In [ ]:
text_splitter=RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=100, chunk_overlap=5)

In [ ]:
doc_splits=text_splitter.split_documents(docs_list)


In [ ]:
vectorstore = Chroma.from_documents(
    documents = doc_splits,
    collection_name = 'rag-chrome',
    embedding = embeddings
)

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
retriever_tool = create_retriever_tool(
    retriever,
    "retrieve_blog_posts",
    "Search and return the information about lilian wang blog posts."
)

In [ ]:
tools = [retriever_tool]

In [ ]:
class AgentState(TypedDict):
  messages : Annotated[Sequence[BaseMessage], add_messages]

In [ ]:
def AI_Assistant(state:AgentState):
  print("----CALL Agent----")
  messages = state["messages"]
  llm_with_tool = llm.bind_tools(tools)
  response = llm_with_tool.invoke(messages)
  return {"messages":[response]}

In [ ]:
def rewrite(state:AgentState):
  print("---Transform Query-----")
  messages = state['messages']
  question = messages[0].content

  msg = [
      HumanMessage(
          content = f""" \n
  look at the input and try to reason about the underlying semantic intent / meaning. \n
  Here is the initital question:
  \n ---------------- \n
  {question}
  \n ---------------- \n
  Formulate an improved question : """,
      )
  ]

  response = llm.invoke(msg)
  return {"messages":[response]}



In [ ]:
def generate(state:AgentState):
  print("----GENERATE-----")
  messages = state["messages"]
  question = messages[0].content
  last_message = messages[-1]
  docs = last_message.content

  prompt = hub.pull('rlm/rag-prompt')

  rag_chain = prompt | llm | StrOutputParser()

  response = rag_chain.invoke({
      "context": docs,
      "question": question
  })

  return {"messages":[response]}

In [ ]:
class grade (BaseModel):
  binary_score: str = Field(description="Relevance Score 'yes' or 'no' ")

In [ ]:
def grade_documents(state:AgentState)->Literal["Output_Generator", "Query_Rewriter"]:
  llm_with_structure_op = llm.with_structured_output(grade)
  prompt = PromptTemplate(
      template = """ You are a grader assessing the relevance of a retreived document to a user question. \n
      Here is the retreived document: \n \n {context} \n \n
      Here is the user question: {question} \n
      If the document contains keyword(s) or semantic meaning related to the user's question, mark it as relevant.
      Give a 'yes' or 'no' answer to show if the document is relevant to the question.""",
      input_variables = ["context","question"]
  )

  chain = prompt | llm_with_structure_op
  messages = state['messages']
  print(f'message from the grader:{messages}')
  last_message = messages[-1]
  question = messages[0].content
  docs = last_message.content
  scored_result = chain.invoke({
      "context": docs,
      "question": question
  })
  score  = scored_result.binary_score

  if score =='yes':
    print('----Decision: DOCS Relevant ----')
    return 'generator'
  else :
    print ('Decision: not relevant')
    return "rewriter"

In [ ]:
workflow = StateGraph(AgentState)
workflow.add_node("ai_assistant",AI_Assistant)
retreiver = ToolNode([retriever_tool])
workflow.add_node("retrieve",retriever)
workflow.add_node("rewriter",rewrite)
workflow.add_node("generator",generate)

In [ ]:
workflow.add_edge(START,'ai_assistant')
workflow.add_conditional_edges("ai_assistant",tools_condition,
                               {"tools":"retrieve",END:END})       # we are using a predefined method called tools_condition which checks the tool condition if YES then retreive otherwise END node

workflow.add_conditional_edges('retrieve',grade_documents,
                               {'rewriter':'rewriter',"generator":'generator'})

workflow.add_edge("rewriter",'ai_assistant')
workflow.add_edge("generator",END)


In [ ]:
app=workflow.compile()

In [ ]:
from IPython.display import Image, display
try:
  display(Image((app.get_graph().draw_mermaid_png())))
except Exception as e:
  print(e)

In [ ]:
app.invoke({"messages": [HumanMessage(content="What is an Autonomous Agent?")]})

## Brief about the code flow .

- First the input comes from the user and goes to the AI - Assistant and branch out to 2 nodes. If the LLM decides to call the tool (i.e retrieval tool here) then the flow will shift to the retreiver node otherwise it will shift to the end node and terminate

- Now in the case of retriver node , it will fetch the relevant documents and then grade them. If relevant documents are found then the flow shift to generate node otherwise it will shift to the rewrite node.

- now in the rewriter node the previous question, is recieved and then it is reframed in hope for a better output and then passed again to the starting node which is the AI_Assistant.
